## Bibliotecas

In [ ]:
import os
import time
import urllib3
import asyncio
import time
import pandas as pd

from googlesearch import search
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, ElementNotInteractableException

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from urllib.parse import urljoin

## Leitura dos dados

In [12]:
arq_cidades = open('cidades.txt', 'r')
cidades = arq_cidades.read().split("\n")
arq_cidades.close()
print(cidades[:10], "\n", cidades[-10:])

['Abadia dos Dourados ', 'Abaeté ', 'Abre Campo ', 'Acaiaca ', 'Açucena ', 'Água Boa ', 'Água Comprida ', 'Aguanil ', 'Águas Formosas ', 'Águas Vermelhas '] 
 ['Vespasiano ', 'Viçosa ', 'Vieiras ', 'Virgem da Lapa ', 'Virgínia ', 'Virginópolis ', 'Virgolândia ', 'Visconde do Rio Branco ', 'Volta Grande ', 'Wenceslau Braz ']


In [13]:
arq = open("links_prefeituras.txt", 'a')
arq.close()

## Gerar link de cada prefeitura

In [14]:
async def gerar_links_prefeituras(cidades):
    
    arq = open("links_prefeituras.txt", 'a')
    
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = False)
    page = await browser.new_page()

    await page.goto("https://www.google.com.br/")

    for cidade in cidades:

        await page.goto("https://www.google.com.br/")
        await page.locator("[aria-label=\"Pesquisar\"]").click()
        await page.locator("[aria-label=\"Pesquisar\"]").fill(cidade + "prefeitura" + " mg")
        await page.locator("[aria-label=\"Pesquisar\"]").press("Enter")
        await page.locator("div.g a").first.click()
        time.sleep(1)
        
        arq.write(page.url)
        arq.write("\n")

    arq.close()
    await page.close()
    await browser.close()

In [15]:
loop = asyncio.get_event_loop()
loop.create_task(gerar_links_prefeituras(cidades[:5]))

In [17]:
arq = open("links_prefeituras.txt", 'r')
links_prefeituras = arq.read().split("\n")
arq.close()
print(links_prefeituras[:5])

['https://abadiadosdourados.mg.gov.br/novo/', 'https://abaete.mg.gov.br/', 'https://abrecampo.mg.gov.br/', 'https://acaiaca.mg.gov.br/', 'https://www.acucena.mg.gov.br/']


## Procurar pelo link do portal a partir do site da prefeitura

In [22]:
async def gerar_links_portais(links_prefeituras):
    
    templates = ["adpmnet", "facil", "memory", "pm-", "portaltp", "portalcidadao",\
                "cidadesmg"]
    
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = False)
    page = await browser.new_page()
    
    urls_portais = []
    templates_portais = []
    
    for url in links_prefeituras:

        temps = set()
        resultados = set()

        try:
            await page.goto(url)
            page_data = await page.content()

        except:
            
            urls_portais.append(list(resultados))
            templates_portais.append(list(temps)) 
            print("Error: " + url)
            continue
            
        soup = BeautifulSoup(page_data, "xml")
        
        for link in soup.find_all('a'):
            
            if("href" in link.attrs):
                url = urljoin(url, str(link.get('href')))

            if url.find("'") != -1:
                continue

            url = url.split("#")[0]
            
            for temp in templates:
                if temp in url:
                    resultados.add(url)
                    temps.add(temp)
            
        urls_portais.append(list(resultados))
        templates_portais.append(list(temps))    
            
    await page.close()
    await browser.close()

    return urls_portais, templates_portais

In [47]:
df = pd.read_csv('tabela.csv', sep=";")
df.head()

,Município,Site Prefeitura,Site Portal,Template
0,Abadia dos Dourados,https://abadiadosdourados.mg.gov.br/novo/,[],[]
1,Abaeté,https://abaete.mg.gov.br/,[],[]
2,Abre Campo,https://abrecampo.mg.gov.br/,"['http://abrecampo-mg.portaltp.com.br/siafic',...","['portaltp', 'pm-']"
3,Acaiaca,https://acaiaca.mg.gov.br/,[],[]
4,Açucena,https://www.acucena.mg.gov.br/,['http://acucena.portalfacil.com.br/transparen...,['facil']


### Teste: usando googlesearch

In [20]:
def gerar_links_prefeituras(cidades):
    
    busca = list(search(cidade + "prefeitura mg" + " ", num_results=5))

    return busca[0]

In [27]:
arq = open("links_prefeituras_search.txt", 'a')

for cidade in cidades:
        
    try:
        arq.write(gerar_links_prefeituras(cidade))
        arq.write("\n")
        
    except:
        
        arq.close()

arq.close()

https://abadiadosdourados.mg.gov.br/novo/
https://abaete.mg.gov.br/
https://abrecampo.mg.gov.br/
https://www.aguacomprida.mg.gov.br/
https://www.aguanil.mg.gov.br/
https://www.aguasformosas.mg.gov.br/
https://www.aguasvermelhas.mg.gov.br/
https://pmsa.mg.gov.br/
https://aiuruoca.mg.gov.br/
https://www.alagoa.mg.gov.br/
http://www.albertina.mg.gov.br/
https://alemparaiba.mg.gov.br/
http://www.alfenas.mg.gov.br/
https://alfredovasconcelos.mg.gov.br/
https://www.almenara.mg.gov.br/
https://pt-br.facebook.com/prefeituradealpercata/
https://alpinopolis.mg.gov.br/
https://www.alterosa.mg.gov.br/
https://www.altocaparao.mg.gov.br/
https://www.altojequitiba.mg.gov.br/
https://www.portal.altoriodoce.mg.gov.br/
https://www.alvarenga.mg.gov.br/servidores
https://pt-br.facebook.com/PrefeituraMunicipaldeAlvinopolis/
https://www.alvoradademinas.mg.gov.br/
https://amparodoserra.mg.gov.br/
http://www.andradas.mg.gov.br/
https://andrelandia.mg.gov.br/site/
https://pt-br.facebook.com/pmAngelandia/
https

KeyboardInterrupt: 